In [32]:
from ctypes import *
from ctypes.wintypes import *

In [33]:
PROCESS_ID = 6612

PLAYERS_START_ADDR = 0x05753000
OFFSET = 0x20
START_ADDR = PLAYERS_START_ADDR + OFFSET

PLAYER_ADDR_STRLEN = 4

PROCESS_VM_READ = 0x0010

PLAYER_CLASS_ID = 0x0133f1a80133ec44

In [34]:
k32 = WinDLL('kernel32')
k32.OpenProcess.argtypes = DWORD,BOOL,DWORD
k32.OpenProcess.restype = HANDLE
k32.ReadProcessMemory.argtypes = HANDLE,LPVOID,LPVOID,c_size_t,POINTER(c_size_t)
k32.ReadProcessMemory.restype = BOOL

In [42]:
def reverse_array(array):
    length = len(array)
    left = 0
    right = length-1
    while left < right:
        temp = array[left]
        array[left] = array[right]
        array[right] = temp
        left = left + 1
        right = right - 1
        
def decode_player(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(PLAYER_ADDR_STRLEN)
    s = c_size_t()
    
    uid_addr = addr + 232
    
    if k32.ReadProcessMemory(process, uid_addr, buf, 4, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1
    
def decode_player_class_id(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(8)
    s = c_size_t()
    
    if k32.ReadProcessMemory(process, addr, buf, 8, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1

In [36]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

number = 0

while True:
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        if value == 0:
            break
        number = number + 1
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break
        
print(number)    

27311


In [37]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        uid = decode_player(player_addr)
        print(uid)
        addr = addr + PLAYER_ADDR_STRLEN
        if uid == -1:
            print("Returned -1, breaking")
            break
    else:
        print("Access Denied!")
        break  

43002215
293783120
293786832
67011241
293788688
67015002
294310696
2000004915
2000004991
294326936
2000005434
294331112
2000005948
2000005982
2000005986
294335288
2000006198
294336680
2000006452
294452216
2000018996
2000019028
2000019467
294478200
2000019511
2000019546
294482376
2000019590
294483304
2000019666
294487016
294487480
2000019961
294488872
294489336
2000019994
2000020014
294490728
294491192
2000020048
2000020073
294491656
294493048
294493512
2000020113
2000020123
294496296
2000020132
294496760
294497688
294498152
294499544
2000020195
2000020205
294501400
2000020247
2000020281
2000020317
2000025182
2000025253
2000025420
2000025475
2000025498
2000025521
2000025754
2000027019
2000027698
294538984
2000027747
2000027818
2000027859
2000028081
2000028144
2000028169
294542232
2000028365
2000028366
2000028369
2000028421
294550120
2000028485
294551512
2000028522
2000028525
2000028540
2000028553
2000028562
2000028568
294554760
2000028600
294558936
294559400
2000028642
2000028650
200002

In [44]:
process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = START_ADDR

num_players = 0
num_other = 0

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        class_id = decode_player_class_id(player_addr)
        
        if class_id == PLAYER_CLASS_ID:
            num_players = num_players + 1
        elif class_id == -1:
            print("Returned -1, breaking")
            break
        else:
            print(addr)
            print(class_id)
            print("---------------")
            num_other = num_other + 1
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break

print(num_players)
print(num_other)

91566116
87506076025937956
---------------
91566120
87506076025937956
---------------
91566128
87506076025937956
---------------
91566136
87506076025937956
---------------
91566148
87506076025937956
---------------
91566156
87506076025937956
---------------
91566172
87506076025937956
---------------
91566180
87506076025937956
---------------
91566188
87506076025937956
---------------
91566204
87506076025937956
---------------
91566216
87506076025937956
---------------
91566224
87506076025937956
---------------
91566232
87506076025937956
---------------
91566236
87506076025937956
---------------
91566244
87506076025937956
---------------
91566248
87506076025937956
---------------
91566260
87506076025937956
---------------
91566264
87506076025937956
---------------
91566276
87506076025937956
---------------
91566280
87506076025937956
---------------
91566284
87506076025937956
---------------
91566296
87506076025937956
---------------
91566304
87506076025937956
---------------
91566308
87

In [41]:
PLAYER_CLASS_ID

86678521727151172

In [3]:
from util.player import player

In [4]:
p = player()

In [5]:
p.x

42